<a href="https://colab.research.google.com/github/ruanraian/1adt-fase1-tech-challenge/blob/tc/tech-challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Enunciado do tech challenge

## O problema

Você é um profissional encarregado de desenvolver um modelo preditivo de regressão para prever o valor dos custos médicos individuais cobrados pelo seguro de saúde.

A base de dados do desafio é o arquivo CSV anexo ao projeto.


## Tarefas

### Exploração de dados:

*   Carregue a base de dados e explore suas características.
*   Analise estatísticas descritiva e visualize distribuições relevantes.

### Pré processamento de dados:

* Realize a limpeza dos dados, tratando valores ausentes se necessário.
* Converta variáveis catégóricas em formatos adequados para modelagem

### Modelagem:

* Crie um modelo preditivo de regressão utilizando uma técnica à sua escolha (por exemplo, Regressão linear, Árvores de decisão, etc).
* Divida o conjunto de dados entre treinamento e teste.

### Treinamento e avaliação do modelo:

* Treine o modelo com conjunto de treinamento.

### Validação estatística:

* Utilize métricas estatísticas para validar a eficácia do modelo (p-value, intervalos de confiança).

### O que avaliaremos:

Apresente resultados visuais, como gráficos de previsões vs. valores reais.
Elabore um relatório que inclua uma análise dos resultados, insights obtidos e validação estatística.

### Observações:

Esperamos que o modelo seja capaz de fazer previsões confiáveis dos custos médicos individuais com base nas características fornecidas.

## Entregável

Como entregável, o grupo deve enbiar um vídeo junto com o link do github do projeto e o código desenvolvido, apresentando o passo a passo do que foi utilizado como a fonte de dados e como os modelos foram criados.

O vídeo deve estar em uma plataforma como Youtube.



# Importando e visualizando os dados


In [1]:
import pandas as pd
dataset = pd.read_csv("insurance.csv")
dataset.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


from matplotlib import pyplot as plt
_df_0['age'].plot(kind='hist', bins=20, title='age')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['bmi'].plot(kind='hist', bins=20, title='bmi')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['children'].plot(kind='hist', bins=20, title='children')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3['charges'].plot(kind='hist', bins=20, title='charges')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_17['smoker'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_17, x='age', y='smoker', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_18['region'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_18, x='age', y='region', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_19['sex'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_19, x='bmi', y='sex', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

## Entendo melhor os dados a serem trabalhados

In [2]:
dataset.shape

(1338, 7)

In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB
